# Exploration des données textuelles (titres et/ou résumés)

Ce notebook explore les données textuelles des résumés et titres de notices bibliographiques afin de voir si des patterns se dégagent rapidement

# Set project

### Packages

In [1]:
# Import librairies
import os
import re
import pandas as pd
import nltk
import numpy as np
import pyLDAvis
import pyLDAvis.sklearn
import seaborn as sns
import spacy
import warnings

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from utils_text_processing import *
from utils_visualization import *

c:\ProgramData\Anaconda3\envs\abes_index\lib\site-packages\flatbuffers\compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [ ]:
# Suppression des FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

### Graphical parameters

In [ ]:
# Parametres graphiques
%matplotlib inline
rc = {
    'font.size': 14,
    'font.family': 'Arial',
    'axes.labelsize': 14,
    'legend.fontsize': 12,
    'axes.titlesize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'figure.max_open_warning': 30}

sns.set(font='Arial', rc=rc)
sns.set_style(
    "whitegrid", {
        'axes.edgecolor': 'k',
        'axes.linewidth': 1,
        'axes.grid': True,
        'xtick.major.width': 1,
        'ytick.major.width': 1
        })
sns.set_context(
    "notebook",
    font_scale=1.1,
    rc={"lines.linewidth": 1.5})
pd.set_option('display.max_columns', None)


In [ ]:
# Autorisation pour la visualisation par pyLDAvis
pyLDAvis.enable_notebook()

### Paths

In [ ]:
# Set paths
path = "."
os.chdir(path)
data_path = path + "\\data"
output_path = path + "\\outputs"
fig_path = path + "\\figs"

### Useful functions

# Import data

In [ ]:
# Input data
working_data_filename = "working_data_sans_dewey.pkl"
analyse_dewey = False

In [ ]:
# Import data
df = pd.read_pickle(os.path.join(data_path, working_data_filename))
print(f"le Fichier de données contient {df.shape[0]} lignes et  {df.shape[1]} colonnes")

le Fichier de données contient 154508 lignes et  9 colonnes


In [ ]:
# Visualisation
df.head()

,PPN,TITRE,RESUME,RAMEAU,DEWEY,DESCR,presence_chaine_indexation,rameau_chaines_index,rameau_concepts
0,000002364,La culture pour vivre,Mort de la culture populaire en France. Mutati...,Culture populaire;Diffusion de la culture;Poli...,840,La culture pour vivre Mort de la culture popul...,False,"[Culture populaire, Diffusion de la culture, P...","[Culture populaire, Diffusion de la culture, P..."
1,000014877,"La nuit, le jour : essai psychanalytique sur l...","Discontinuité, latence, rétablissement d’une c...",Complexe de castration;Psychanalyse;Rêves,154.63,"La nuit, le jour : essai psychanalytique sur l...",False,"[Complexe de castration, Psychanalyse, Rêves]","[Complexe de castration, Psychanalyse, Rêves]"
2,000021857,"Ruptures, cultures","Il faut imaginer Robinson sur son île, au mome...",Culture,840,"Ruptures, cultures Il faut imaginer Robinson s...",False,[Culture],[Culture]
3,00002564X,La révolution structurale,"Mutations ou crises, les brusques accès de fiè...",Structuralisme,100,"La révolution structurale Mutations ou crises,...",False,[Structuralisme],[Structuralisme]
4,000026352,La Destruction du temple,"Oswald tire sur Kennedy. Jusque-là, c'est bon,...",Science-fiction américaine -- Traductions fran...,830,La Destruction du temple Oswald tire sur Kenne...,True,[Science-fiction américaine -- Traductions fra...,"[Science-fiction américaine, Traductions franç..."


# Exploration des titres

In [ ]:
# List des stopwords
list(set(stopwords.words("french")))

['sa',
 'c',
 'mais',
 'même',
 'fussiez',
 'eût',
 'les',
 'l',
 'du',
 's',
 'furent',
 'tes',
 'que',
 'serait',
 'nos',
 'je',
 'eûtes',
 'fusse',
 'ce',
 'et',
 'aurons',
 'ils',
 'ces',
 'est',
 'serez',
 'notre',
 'suis',
 'ayant',
 'aient',
 'lui',
 'me',
 'tu',
 'm',
 'j',
 'fussent',
 'ayants',
 'soit',
 'sois',
 'un',
 'votre',
 'eussiez',
 'étaient',
 'étante',
 'aura',
 'ait',
 'mon',
 'auriez',
 'ne',
 'pour',
 'serions',
 'aie',
 'fûmes',
 'étiez',
 'sur',
 'avez',
 'aurez',
 'soient',
 'ont',
 'eusse',
 'n',
 'au',
 'aurais',
 'étantes',
 'leur',
 'fusses',
 'étants',
 'eux',
 'ma',
 'mes',
 'fut',
 'qu',
 'auraient',
 'se',
 'seriez',
 'eurent',
 'il',
 'eue',
 'ses',
 'qui',
 'êtes',
 'avons',
 'avait',
 'la',
 'pas',
 'ayantes',
 'ta',
 't',
 'étées',
 'te',
 'soyons',
 'as',
 'soyez',
 'vous',
 'le',
 'en',
 'étant',
 'fus',
 'ayons',
 'moi',
 'avais',
 'elle',
 'sont',
 'auras',
 'd',
 'auront',
 'ou',
 'fût',
 'eues',
 'étions',
 'ayez',
 'aurai',
 'eusses',
 'une

In [ ]:
# Test function
idx = 698
text = df.loc[idx, 'TITRE']
print(text)

Le voyage en Russie : anthologie des voyageurs français aux XVIIIe et XIXe siècles


In [ ]:
# Exemple lemmatization
doc = nlp(text)
print("Sans lemmatization :", doc)

empty_list = []
for token in doc:
    empty_list.append(token.lemma_)

final_string = ' '.join(map(str,empty_list))
print("Après lemmatization :",final_string)

Sans lemmatization : Le voyage en Russie : anthologie des voyageurs français aux XVIIIe et XIXe siècles
Après lemmatization : le voyage en Russie : anthologie de voyageur français à xviii et xix siècle


In [ ]:
process = PreprocessData(
    df = df,
    input_col="TITRE",
    output_col="TITRE_processed",
    add_words = [],
    numeric = False,
    stopw = True,
    stem = True,
    lem = False)

In [ ]:
process.df

,PPN,TITRE,RESUME,RAMEAU,DEWEY,DESCR,presence_chaine_indexation,rameau_chaines_index,rameau_concepts,TITRE_processed
0,000002364,La culture pour vivre,Mort de la culture populaire en France. Mutati...,Culture populaire;Diffusion de la culture;Poli...,840,La culture pour vivre Mort de la culture popul...,False,"[Culture populaire, Diffusion de la culture, P...","[Culture populaire, Diffusion de la culture, P...","[cultur, vivr]"
1,000014877,"La nuit, le jour : essai psychanalytique sur l...","Discontinuité, latence, rétablissement d’une c...",Complexe de castration;Psychanalyse;Rêves,154.63,"La nuit, le jour : essai psychanalytique sur l...",False,"[Complexe de castration, Psychanalyse, Rêves]","[Complexe de castration, Psychanalyse, Rêves]","[nuit, jour, essai, psychanalyt, fonction, men..."
2,000021857,"Ruptures, cultures","Il faut imaginer Robinson sur son île, au mome...",Culture,840,"Ruptures, cultures Il faut imaginer Robinson s...",False,[Culture],[Culture],"[ruptur, cultur]"
3,00002564X,La révolution structurale,"Mutations ou crises, les brusques accès de fiè...",Structuralisme,100,"La révolution structurale Mutations ou crises,...",False,[Structuralisme],[Structuralisme],"[révolu, structural]"
4,000026352,La Destruction du temple,"Oswald tire sur Kennedy. Jusque-là, c'est bon,...",Science-fiction américaine -- Traductions fran...,830,La Destruction du temple Oswald tire sur Kenne...,True,[Science-fiction américaine -- Traductions fra...,"[Science-fiction américaine, Traductions franç...","[destruct, templ]"
...,...,...,...,...,...,...,...,...,...,...
169658,26899885X,Blablabla : en finir avec le bavardage climatique,Dans le brouhaha de nos conversations et de l'...,Réchauffement de la Terre;Écologie,,Blablabla : en finir avec le bavardage climati...,False,"[Réchauffement de la Terre, Écologie]","[Réchauffement de la Terre, Écologie]","[blablabl, fin, bavardag, climat]"
169659,268998884,Politique de transition écologique : Démocrati...,La transition écologique est devenue une préoc...,Aménagement du territoire;Marchés publics;Tran...,,Politique de transition écologique : Démocrati...,True,"[Aménagement du territoire, Marchés publics, T...","[Aménagement du territoire, Marchés publics, T...","[polit, transit, écolog, démocrat, droit, financ]"
169660,268999066,Abécédaire : mots et rites d'ailleurs,Je dédie cet abécédaire des concepts des cultu...,Ethnopsychiatrie;Psychanalyse et ésotérisme;Ri...,,Abécédaire : mots et rites d'ailleurs Je dédie...,False,"[Ethnopsychiatrie, Psychanalyse et ésotérisme,...","[Ethnopsychiatrie, Psychanalyse et ésotérisme,...","[abécédair, mot, rit, d'ailleur]"
169661,268999503,"Consommez moins, consommez mieux",Un cahier pratique et ludique à remplir soi-mê...,Biens de consommation durables;Consommation --...,332.024,"Consommez moins, consommez mieux Un cahier pra...",True,"[Biens de consommation durables, Consommation ...","[Biens de consommation durables, Consommation,...","[consomm, consomm, mieux]"


In [ ]:
# add words
add_words = [
        "la",
        "de",
        "le",
        "les",
        "l",
        "au",
        "du"
]

process = PreprocessData(
    df = df,
    input_col="TITRE",
    output_col="TITRE_processed",
    add_words = add_words,
    encod = None,
    numeric = False,
    stopw = True,
    stem = False,
    lem = True)

In [ ]:
# Get df
df = process.df

In [ ]:
# Exemple
idx = 300
print("Titre brut: \n", df['TITRE'].iloc[idx])
print("\nTitre après processing :\n", df['TITRE_processed'].iloc[idx])

Titre brut: 
 Origines et symbolisme des productions textiles : de la toile et du fil

Titre après processing :
 ['origine', 'symbolism', 'production', 'textile', 'toile', 'fil']


In [17]:
plot_wordcloud(df['TITRE_processed'])

TypeError: unhashable type: 'list'

<Figure size 1500x800 with 0 Axes>

## Exploration des topics (pyLDAvis)

In [18]:
# Define vectorization
vectorizer = TfidfVectorizer(
    max_features=300,
    ngram_range=(1, 5),
    min_df=10,
    max_df=0.95)

In [19]:
# Define vizualisation
n_comp = 15
lda = LatentDirichletAllocation(
    n_components=n_comp,
    learning_method='online',
    random_state=42
    )

In [20]:
# Vectorization
feature = 'TITRE_processed'
cv_transform = vectorizer.fit_transform(df[feature])
print("Dimensions de la matrice", cv_transform.shape)

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
# Reduction dimension
x_red = lda.fit_transform(cv_transform)

In [ ]:
# Visualisation
p = pyLDAvis.sklearn.prepare(lda, cv_transform, vectorizer)
pyLDAvis.save_html(p, os.path.join(fig_path, "pyldavis_titres_lemma.html"))

# Exploration des résumés

In [ ]:
# Preproces des résumés
process = PreprocessData(
    df = df,
    input_col="RESUME",
    output_col="RESUME_processed",
    add_words=add_words,
    numeric=False,
    stopw=True,
    stem=False,
    lem=True)

In [ ]:
# Get df
df = process.df

In [ ]:
# Exemple
idx = 12945
print("Résumé brut: \n", df['RESUME'].iloc[idx])
print("\nRésumé après processing :\n", df['RESUME_processed'].iloc[idx ])

In [ ]:
plot_wordcloud(df['RESUME_processed'])

In [ ]:
df

In [ ]:
# Vectorization
feature = 'RESUME_processed'
cv_transform = vectorizer.fit_transform(df[feature])
print("Dimensions de la matrice", cv_transform.shape)

In [ ]:
# Reduction dimension
x_red = lda.fit_transform(cv_transform)

In [ ]:
# Vizualisation
p = pyLDAvis.sklearn.prepare(lda, cv_transform, vectorizer)
pyLDAvis.save_html(p, os.path.join(fig_path, "pyldavis_resumes_lemma.html"))

# Exploration des description (titre+ resumé)

In [ ]:
# Preproces des descriptions
process = PreprocessData(
    df = df,
    input_col="DESCR",
    output_col="DESCR_processed",
    add_words=add_words,
    numeric=False,
    stopw=True,
    stem=False,
    lem=True)

In [ ]:
# Get df
df = process.df

In [ ]:
# Exemple
idx = 6549
print("Description brute: \n", df['DESCR'].iloc[idx])
print("\nDescription après processing :\n", df['DESCR_processed'].iloc[idx])

In [ ]:
plot_wordcloud(df['DESCR_processed'])

In [ ]:
# Vectorization
feature = 'DESCR_processed'
cv_transform = vectorizer.fit_transform(df[feature])
print("Dimensions de la matrice", cv_transform.shape)

In [ ]:
# Reduction dimension
x_red = lda.fit_transform(cv_transform)

In [ ]:
# Vizualisation
p = pyLDAvis.sklearn.prepare(lda, cv_transform, vectorizer)
pyLDAvis.save_html(p, os.path.join(fig_path, "pyldavis_description_lemma.html"))